In [17]:
import pandas as pd
import numpy as np

In [18]:
# Try using ISO-8859-1
try:
    df = pd.read_csv('Ecommerce_data.csv', encoding='ISO-8859-1')
    print(df.head())
except Exception as e:
    print(e)

# If ISO-8859-1 doesn't work, you can try Windows-1252
try:
    df = pd.read_csv('Ecommerce_data.csv', encoding='windows-1252')
    print(df.head())
except Exception as e:
    print(e)


  customer_id customer_first_name customer_last_name    category_name  \
0  C_ID_45866                Mary             Fuller  Office Supplies   
1  C_ID_44932                Alan            Edelman  Office Supplies   
2  C_ID_70880                Mary             Gayman  Office Supplies   
3  C_ID_33157             Raymond              Eason  Office Supplies   
4  C_ID_58303                Mary           Gonzalez        Furniture   

                           product_name customer_segment customer_city  \
0                            Xerox 1913        Corporate  New Rochelle   
1    #6 3/4 Gummed Flap White Envelopes        Corporate       Houston   
2       Belkin 8 Outlet Surge Protector         Consumer    Louisville   
3  GBC VeloBinder Manual Binding System        Corporate       Chicago   
4         Eldon Pizzaz Desk Accessories      Home Office  Philadelphia   

  customer_state customer_country customer_region  ...  order_date  \
0       New York    United States            E

In [19]:
df.head()

,customer_id,customer_first_name,customer_last_name,category_name,product_name,customer_segment,customer_city,customer_state,customer_country,customer_region,...,order_date,order_id,ship_date,shipping_type,days_for_shipment_scheduled,days_for_shipment_real,order_item_discount,sales_per_order,order_quantity,profit_per_order
0,C_ID_45866,Mary,Fuller,Office Supplies,Xerox 1913,Corporate,New Rochelle,New York,United States,East,...,11-05-2022,O_ID_3001072,11-07-2022,Second Class,2,2,35.0,500.0,5,223.199997
1,C_ID_44932,Alan,Edelman,Office Supplies,#6 3/4 Gummed Flap White Envelopes,Corporate,Houston,Texas,United States,Central,...,20-06-2022,O_ID_3009170,23-06-2022,Second Class,2,3,85.0,500.0,5,199.199997
2,C_ID_70880,Mary,Gayman,Office Supplies,Belkin 8 Outlet Surge Protector,Consumer,Louisville,Kentucky,United States,South,...,25-06-2022,O_ID_3047567,30-06-2022,Standard Class,4,5,75.0,44.0,5,195.500000
3,C_ID_33157,Raymond,Eason,Office Supplies,GBC VeloBinder Manual Binding System,Corporate,Chicago,Illinois,United States,Central,...,10-06-2022,O_ID_3060575,10-10-2022,Second Class,2,4,60.0,254.0,1,220.000000
4,C_ID_58303,Mary,Gonzalez,Furniture,Eldon Pizzaz Desk Accessories,Home Office,Philadelphia,Pennsylvania,United States,East,...,02-05-2022,O_ID_3064311,08-01-2022,First Class,1,2,125.0,500.0,1,97.500000


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113270 entries, 0 to 113269
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   customer_id                  113270 non-null  object 
 1   customer_first_name          113270 non-null  object 
 2   customer_last_name           113270 non-null  object 
 3   category_name                113270 non-null  object 
 4   product_name                 113270 non-null  object 
 5   customer_segment             113270 non-null  object 
 6   customer_city                113270 non-null  object 
 7   customer_state               113270 non-null  object 
 8   customer_country             113270 non-null  object 
 9   customer_region              113270 non-null  object 
 10  delivery_status              113270 non-null  object 
 11  order_date                   113270 non-null  object 
 12  order_id                     113270 non-null  object 
 13 

In [32]:
#unique categories count
df['customer_id'].nunique()

42047

In [23]:
def refine_gender(name):
    common_female_endings = ('a', 'e', 'i', 'y', 'l')
    common_male_endings = ('n', 'o', 'r', 's', 't', 'd', 'm')

    if name[-1].lower() in common_female_endings:
        return 'Female'
    elif name[-1].lower() in common_male_endings:
        return 'Male'
    else:
        # Making an educated guess if the name ends with other letters
        if len(name) > 6:  # Assuming longer names might be more feminine
            return 'Female'
        else:
            return 'Male'
df['gender'] = df['customer_first_name'].apply(refine_gender)

In [27]:
def age_assignment(segment, category):
    # Example adjustments for age based on segment and category - these can be refined
    segment_age_adjustment = {
        'Consumer': 0,  # Base age range for consumers
        'Corporate': 5,  # Assuming corporate customers might be slightly older on average
        'Home Office': 3  # Home office might indicate a mid-range age
    }
    category_age_adjustment = {
        'Technology': 5,  # Technology enthusiasts might be younger
        'Furniture': 10,  # Furniture might indicate older customers, possibly homeowners
        'Office Supplies': 0  # No specific age adjustment for office supplies
    }
    base_age = 25  # Starting point for age
    age = base_age + segment_age_adjustment.get(segment, 0) + category_age_adjustment.get(category, 0)
    return np.random.randint(age, age + 20)
df['age'] = df.apply(lambda row: age_assignment(row['customer_segment'], row['category_name']), axis=1)

In [24]:
import random

def assign_churn(row):
    # Factors that might influence churn, adjust weights as needed
    churn_probability = 0.1  # Base probability of churn
    if row['days_for_shipment_real'] > row['days_for_shipment_scheduled']:
        churn_probability += 0.1  # Late delivery increases churn probability
    if row['order_item_discount'] < 20:
        churn_probability += 0.05  # Low discounts might increase churn
    if row['profit_per_order'] < 0:
        churn_probability += 0.05  # Negative profit orders might lead to unsatisfied customers
    
    return random.choices([1, 0], [churn_probability, 1 - churn_probability])[0]

df['churn'] = df.apply(assign_churn, axis=1)

In [25]:
import random

category_stock_levels = {
    'Office Supplies': (50, 200),  # Range of stock levels for Office Supplies
    'Furniture': (5, 50),  # Range of stock levels for Furniture
    'Technology': (10, 100)  # Range of stock levels for Technology
}

def generate_stock_data(row):
    category = row['category_name']
    min_stock, max_stock = category_stock_levels.get(category, (0, 100))  # Default range if category not found
    
    # Factors influencing stock levels
    if row['order_quantity'] > 3:
        max_stock -= 20  # High order quantities might deplete stock faster
    
    # Generate stock level within the range for the category
    stock_level = random.randint(min_stock, max_stock)
    
    return stock_level

df['stock_level'] = df.apply(generate_stock_data, axis=1)

In [28]:
df.head()

,customer_id,customer_first_name,customer_last_name,category_name,product_name,customer_segment,customer_city,customer_state,customer_country,customer_region,...,days_for_shipment_scheduled,days_for_shipment_real,order_item_discount,sales_per_order,order_quantity,profit_per_order,gender,churn,stock_level,age
0,C_ID_45866,Mary,Fuller,Office Supplies,Xerox 1913,Corporate,New Rochelle,New York,United States,East,...,2,2,35.0,500.0,5,223.199997,Female,0,171,35
1,C_ID_44932,Alan,Edelman,Office Supplies,#6 3/4 Gummed Flap White Envelopes,Corporate,Houston,Texas,United States,Central,...,2,3,85.0,500.0,5,199.199997,Male,0,60,47
2,C_ID_70880,Mary,Gayman,Office Supplies,Belkin 8 Outlet Surge Protector,Consumer,Louisville,Kentucky,United States,South,...,4,5,75.0,44.0,5,195.500000,Female,1,119,28
3,C_ID_33157,Raymond,Eason,Office Supplies,GBC VeloBinder Manual Binding System,Corporate,Chicago,Illinois,United States,Central,...,2,4,60.0,254.0,1,220.000000,Male,0,76,32
4,C_ID_58303,Mary,Gonzalez,Furniture,Eldon Pizzaz Desk Accessories,Home Office,Philadelphia,Pennsylvania,United States,East,...,1,2,125.0,500.0,1,97.500000,Female,1,43,46
